In [1]:
import pandas as pd
import numpy as np

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords 
from pymystem3 import Mystem
from string import punctuation

[nltk_data] Downloading package stopwords to /home/danat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Читаем файл:
df = pd.read_csv('Wordlist.csv', header = None, encoding = 'utf-8')
df

,0,1,2
0,Коктейли,Алкоголь,Вино
1,NaN,Веселье,Друзья
2,NaN,Отдых,На природе
3,Барная стойка,Стаканы,Вода
4,NaN,Танцы,Бальные
5,NaN,Беседы,По душам
6,Бармены,Коктейли,Вкусные
7,NaN,Чаевые,Вознаграждение
8,NaN,Стойка,За баром
9,Люди,Общение,Эмоции


In [3]:
#Создаём лемматизатор и список стоп-слов:
m = Mystem()
russian_stopwords = stopwords.words("russian")

In [4]:
#Функция для предобработки текста:
def preprocess_text(text):
    
    tokens = m.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [5]:
#преобразуем слова в их исходные формы:
df = df.fillna("")
df = df.applymap(preprocess_text)

In [6]:
#Считаем частотки уникальных слов в каждом столбце и оставляем только те, что встречались как мин. дважды:
pervich = df[0].value_counts()
pervich = pervich.loc[pervich > 1]

vtorich = df[1].value_counts()
vtorich = vtorich.loc[vtorich > 1]

tretich = df[2].value_counts()
tretich = tretich.loc[tretich > 1]

In [7]:
#записываем получившийся список частот в csv-таблицу:
word_freqs = pd.concat([pervich, vtorich, tretich], axis=1)
word_freqs.replace(np.nan, '', inplace = True, regex = True)
word_freqs.columns = ['Первичные', 'Вторичные', 'Третичные']
word_freqs.to_csv('Word_freqs.csv', sep=';', encoding='cp1251')

/home/danat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


In [8]:
#заполняем пропуски в 1-м столбце таблицы (для будущего графа):
for i in range(0, len(df[0]) - 2, 3):
    df[0][i + 1] = df[0][i]
    df[0][i + 2] = df[0][i]

In [9]:
print('Пожалуйста, введите слово(фразу), к которому(-ой) строится карта ассоциаций')
place = input()

Пожалуйста, введите слово(фразу), к которому(-ой) строится карта ассоциаций
слово


In [10]:
#создаём матрицу для графа в Gephi:
df0 = df.copy()
df0 = df0.drop([1, 2], axis = 1)
place = [place] * len(df0)
df0.insert(loc = 0, column = 'Исходное слово', value = place)
df1 = df.copy()
df1 = df1.drop([0], axis = 1)
df = df.drop([2], axis = 1)
df.columns = df0.columns
df1.columns = df.columns
df.index = range(len(df0.index), 2 * len(df0.index))
df1.index = range(2 * len(df.index), 3 * len(df.index))
gra = df0.append(df)
graph = gra.append(df1)

In [11]:
#записываем получившуюся матрицу (для графа) в csv-таблицу:
graph.to_csv('For_graph.csv', sep=';', header = False, index = False, encoding='cp1251')